In [ ]:
#@title Aprendizaje Profundo | Otoño 2021 by Datitos{display-mode: "form" }
#@markdown ![71335171.png](data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAMgAAADICAIAAAAiOjnJAAACwElEQVR4nOzdMY7iQBBA0WU197/FnJNNJ/FqWvLHZfd7McIGfVVQos3X+/3+A2f7e/UN8EzCIiEsEsIiISwSwiIhLBLCIiEsEsIiISwSwiIhLBLCIiEsEsIiISwSwiIhLBLCIiEsEsIiISwSwiIhLBLCIiEsEsIiISwSwiIhLBLCIiEsEsIiISwSwiIhLBLCIiEsEsIiISwSwiIhLBJfV9/A/7xer6XX3/1/gZ70eU0sEsIiISwSwiIhLBLCIiEsEiP2WEf7m9U9zVnvU9vh85pYJIRFQlgkhEVCWCSERUJYJEbssc5ytL+5at8zec9UM7FICIuEsEgIi4SwSAiLhLBIPGqPdWR1v1VfdwcmFglhkRAWCWGREBYJYZEQFokt9lhHdt4z1UwsEsIiISwSwiIhLBLCIiEsEh/dY+18zq4w7RzlTyYWCWGREBYJYZEQFglhkRAWiS1+jzVhr/Mbd7nP3zCxSAiLhLBICIuEsEgIi4SwSDxqj1U/72r1ulc9l2sCE4uEsEgIi4SwSAiLhLBICIvE7D3W9/fSy6/63dLqdZfvc/F7mMDEIiEsEsIiISwSwiIhLBLCIjF7j3WSs87rTXufyUwsEsIiISwSwiIhLBLCIiEsEq8Ju5N6r3OXc3xP2oeZWCSERUJYJIRFQlgkhEVCWCRG7LGOTN7TfNIdvwcTi4SwSAiLhLBICIuEsEgIi8Toc4U7Pyf9p8n7qiMmFglhkRAWCWGREBYJYZEQFonRe6wjd9zr7MbEIiEsEsIiISwSwiIhLBLCIiEsEsIiISwSwiIhLBLCIiEsEsIiISwSwiIhLBLCIiEsEsIiISwSwiIhLBLCIiEsEsIiISwSwiIhLBLCIiEsEsIiISwSwiIhLBLCIiEsEsIiISwSwiIhLBLCIiEsEsIiISwSwiIhLBLCIiEsEsIi8S8AAP//HtRtH09JwIEAAAAASUVORK5CYII=)
#El siguiente notebook fue traducido por Pablo Marinozi como el cuarto trabajo práctico correspondiente a la versión de Otoño del 2021 del curso Aprendizaje Profundo organizado por Datitos
#El tutorial original fue diseñado por Ben Trevett y fue publicado en su github https://github.com/bentrevett
#Para mayor información consultar https://datitos.github.io/curso-aprendizaje-profundo/#calendario

# Trabajo Práctico N°6: Clasificación usando Redes Neuronales Recurrentes

# Sección 1 - Análisis de sentimiento simple

En este práctico, crearemos un modelo de aprendizaje automático para detectar sentimientos (es decir, detectar si una oración es positiva o negativa) usando PyTorch y TorchText. Esto se hará en reseñas de películas, utilizando el [conjunto de datos de IMDb](http://ai.stanford.edu/~amaas/data/sentiment/).

Comenzaremos de manera muy simple para comprender los conceptos generales sin preocuparnos realmente por los buenos resultados. Las siguientes secciones se basarán en estos conceptos y obtendremos buenos resultados.

### Introducción

Usaremos una **red neuronal recurrente** (RNN), ya que se usan comúnmente en el análisis de secuencias. Un RNN toma una secuencia de palabras, $X = \{x_1, ..., x_T \}$, una a la vez, y produce un _estado oculto_, $h$, para cada palabra. Usamos el RNN _recurrentemente_ introduciendo la palabra actual $x_t$ así como el estado oculto de la palabra anterior, $h_{t-1}$, para producir el siguiente estado oculto, $h_t$.

$$h_t = \text{RNN}(x_t, h_{t-1})$$

Una vez que tenemos nuestro estado oculto final, $h_T$, (a partir de la última palabra en la secuencia, $x_T$) lo alimentamos a través de una capa lineal, $f$, (también conocida como capa densa), para recibir nuestro sentimiento predicho, $\hat{y} = f(h_T)$.

A continuación se muestra una oración de ejemplo, con la RNN prediciendo cero, lo que indica un sentimiento negativo. La RNN se muestra en naranja y la capa lineal se muestra en plateado. Tenga en cuenta que usamos la misma RNN para cada palabra, es decir, tiene los mismos parámetros. El estado oculto inicial, $ h_0 $, es un tensor inicializado a todos ceros.

![](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/assets/sentiment1.png?raw=1)

**Nota:** algunas capas y pasos se han omitido del diagrama, pero se explicarán más adelante.

## Preparando los datos

Uno de los conceptos principales de TorchText es el "Field". Estos definen cómo se deben procesar tus datos. En nuestra tarea de clasificación de sentimientos, los datos consisten en la cadena sin procesar de la reseña y la etiqueta, ya sea "pos" o "neg".

Los parámetros de un "Field" especifican cómo se deben procesar los datos.

Usamos el campo `TEXT` para definir cómo se debe procesar la reseña y el campo` LABEL` para procesar la opinión.

Nuestro Field `TEXT` tiene` tokenize = 'spacy'` como argumento. Esto define que la "tokenización" (el acto de dividir la cadena en "tokens" discretos) debe realizarse utilizando el tokenizador [spaCy](https://spacy.io). Si no se pasa ningún argumento `tokenize`, el valor predeterminado es simplemente dividir la cadena en espacios. También necesitamos especificar un `tokenizer_language` que le dice a torchtext qué modelo de spaCy usar. Usamos el modelo `en_core_web_sm` que debe descargarse con ` python -m spacy download en_core_web_sm` antes de ejecutar este notebook.

"LABEL" se define mediante un "LabelField", un subconjunto especial de la clase "Field" que se utiliza específicamente para manipular etiquetas. Explicaremos el argumento `dtype` más adelante.

Para obtener más información sobre `Fields`, vaya [aquí](https://github.com/pytorch/text/blob/master/torchtext/data/field.py).

También establecemos las semillas aleatorias para la reproducibilidad.

In [ ]:
import torch
from torchtext.legacy import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm')
LABEL = data.LabelField(dtype = torch.float)

Otra característica útil de TorchText es que tiene soporte para conjuntos de datos comunes utilizados en el procesamiento del lenguaje natural (NLP).

El siguiente código descarga automáticamente el conjunto de datos de IMDb y lo divide en las divisiones canónicas de entrenamiento/prueba como objetos `torchtext.datasets`. Procesa los datos utilizando los `Fields` que hemos definido previamente. El conjunto de datos de IMDb consta de 50.000 reseñas de películas, cada una etiquetada como reseña positiva o negativa.

In [ ]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

KeyboardInterrupt: ignored

La celda anterior tarda mucho tiempo en ejecutarse debido a que el método de tokenización spaCy es bastante lento. Por eso vamos a guardar los datos tokenizados en archivos para cargarlos de disco cada vez que queramos usarlos.

In [ ]:
import json

train_examples = [vars(t) for t in train_data]
test_examples = [vars(t) for t in test_data]

with open('train.json', 'w+') as f:
    for example in train_examples:
        json.dump(example, f)
        f.write('\n')
        
with open('test.json', 'w+') as f:
    for example in test_examples:
        json.dump(example, f)
        f.write('\n')

Para cargar los datos desde disco, usamos el siguiente código.

In [ ]:
TEXT = data.Field()
LABEL = data.LabelField(dtype = torch.float)

fields = {'text': ('text', TEXT), 'label': ('label', LABEL)}

train_data, test_data = data.TabularDataset.splits(
    path = '/content',
    train = 'train.json',
    test = 'test.json',
    format = 'json',
    fields = fields
)

Podemos ver cuántos ejemplos hay en cada división comprobando su longitud.

In [ ]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 13632
Number of testing examples: 25000


También podemos comprobar un ejemplo.

In [ ]:
print(vars(train_data.examples[0]))

El conjunto de datos de IMDb solo tiene divisiones de entrenamiento/prueba, por lo que necesitamos crear un conjunto de validación. Podemos hacer esto con el método `.split()`.

De forma predeterminada, esto divide 70/30, sin embargo, al pasar un argumento `split_ratio`, podemos cambiar la proporción de la división, es decir, un` split_ratio` de 0.8 significaría que el 80% de los ejemplos componen el conjunto de entrenamiento y el 20% componen el conjunto de validación.

También pasamos nuestra semilla aleatoria al argumento `random_state`, asegurándonos de obtener la misma división de entrenamiento/validación cada vez que lo ejecutemos.

In [ ]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

Nuevamente, vamos a ver cuántos ejemplos hay en cada división.

In [ ]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


A continuación, tenemos que construir un _vocabulario_. Ésta es efectivamente una tabla de búsqueda donde cada palabra única en su conjunto de datos tiene un _indice_ correspondiente (un número entero).

Hacemos esto porque nuestro modelo de aprendizaje automático no puede operar con cadenas, solo con números. Cada _índice_ se utiliza para construir un vector _one-hot_ para cada palabra. Un vector one-hot es un vector donde todos los elementos son 0, excepto uno, que es 1, y la dimensionalidad es el número total de palabras únicas en su vocabulario, comúnmente denotado por $ V $.

![](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/assets/sentiment5.png?raw=1)

La cantidad de palabras únicas en nuestro conjunto de entrenamiento es de más de 100.000, lo que significa que nuestros vectores one-hot tendrán más de 100.000 dimensiones. Esto hará que el entrenamiento sea lento y posiblemente no se ajuste a su GPU (si está usando una).

Hay dos formas de reducir nuestro vocabulario de manera efectiva: solo podemos tomar las $n$ palabras más comunes o ignorar las palabras que aparecen menos de $m$ veces. Haremos lo primero, manteniendo solo las 25.000 palabras más usadas en el dataset.

¿Qué hacemos con las palabras que aparecen en los ejemplos pero que hemos sacado del vocabulario? Los reemplazamos con un token especial _unknown_ o ``. Por ejemplo, si la oración fuera "This film is great and I love it", pero la palabra "love" no estaba en el vocabulario, se convertiría en "This film is great and I <unk> it".

La siguiente celda construye el vocabulario, manteniendo solo los tokens `max_size` más comunes.

In [ ]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

¿Por qué construimos el vocabulario usando solo el conjunto de entrenamiento? Al probar cualquier sistema de aprendizaje automático, no tenemos que mirar el conjunto de prueba de ninguna manera. Tampoco incluimos el conjunto de validación porque queremos que refleje el conjunto de prueba tanto como sea posible.

In [ ]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


¿Por qué el tamaño de vocabulario es 25002 y no 25000? Uno de los tokens añadidos es el token `<unk>` y el otro es un token `<pad>`.

Cuando ingresamos oraciones en nuestro modelo, ingresamos un _lote_ de ellas a la vez, es decir, más de una a la vez, y todas las oraciones del lote deben tener el mismo tamaño. Por lo tanto, para garantizar que cada oración del lote sea del mismo tamaño, se rellena cualquier frase más corta que la más larga del lote con tokens `<pad>`.

![](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/assets/sentiment6.png?raw=1)

También podemos ver las palabras más comunes del vocabulario y sus frecuencias.

In [ ]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 200156), ('a', 108181), ('and', 106418), ('of', 99511), ('to', 92452), ('is', 72025), ('in', 59506), ('I', 46002), ('that', 45089), ('this', 39889), ('it', 38102), ('/><br', 35889), ('was', 32750), ('as', 29657), ('with', 28966), ('for', 28697), ('The', 23621), ('but', 23587), ('on', 21514), ('movie', 21232)]


También podemos ver el vocabulario directamente usando el método `stoi`(**s**tring **to** **i**nt) o el método `itos` (**i**nt **to**  **s**tring).

In [ ]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', 'a', 'and', 'of', 'to', 'is', 'in', 'I']


También podemos verificar las etiquetas, asegurándonos de que 0 sea negativo y 1 sea positivo.

In [ ]:
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


El paso final de la preparación de los datos es la creación de los iteradores. Repetimos estos en el ciclo de entrenamiento/evaluación, y devuelven un lote de ejemplos (indexados y convertidos en tensores) en cada iteración.

Usaremos un `BucketIterator` que es un tipo especial de iterador que devolverá un lote de ejemplos donde cada ejemplo tiene una longitud similar, minimizando la cantidad de tokens de relleno.

También queremos colocar los tensores devueltos por el iterador en la GPU (si está usando uno). PyTorch maneja esto usando `torch.device`, luego pasamos este dispositivo al iterador.

In [ ]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort=False,
    device = device)

## Construcción del modelo

La siguiente etapa es construir el modelo que eventualmente vamos a entrenar y evaluar.

Nuestra clase `RNN` es una subclase de` nn.Module`.

Dentro del `__init__` definimos las _capas_ del módulo. Nuestras tres capas son una capa _embedding_, nuestra RNN y una capa _densa_. Todas las capas tienen sus parámetros inicializados a valores aleatorios, a menos que se especifique explícitamente.

La capa de embedding se utiliza para transformar nuestro vector ralos one-hot (ralos ya que la mayoría de los elementos son 0) en un vector de embedding denso (denso ya que la dimensionalidad es mucho más pequeña y todos los elementos son números reales). Esta capa de embedding es simplemente una capa densa única. Además de reducir la dimensionalidad de la entrada al RNN, existe la teoría de que las palabras que tienen un impacto similar en el sentimiento de la revisión se mapean juntas en este denso espacio vectorial. Para obtener más información sobre los embeddings de palabras, consulte [aquí](https://monkeylearn.com/blog/word-embeddings-transform-text-numbers/).

La capa RNN es nuestra RNN que toma nuestro vector denso y el estado oculto anterior $h_{t-1}$, y lo usa para calcular el siguiente estado oculto, $ h_t $.

![](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/assets/sentiment7.png?raw=1)

Finalmente, la capa linear toma el estado oculto final y lo alimenta a través de una capa densa, $ f(h_T) $, transformándola a la dimensión de salida correcta.

Se llama al método `forward` cuando introducimos ejemplos en nuestro modelo.

Cada lote, `text`, es un tensor de tamaño **[longitud de la oración, tamaño del lote]**. Es un lote de oraciones, cada una de las cuales tiene cada palabra convertida en un vector one-hot.

Puede notar que este tensor debería tener otra dimensión debido a los vectores one-hot, sin embargo PyTorch almacena convenientemente un vector one-hot como su valor de índice, es decir, el tensor que representa una oración es solo un tensor de los índices para cada token en ese oración. El acto de convertir una lista de tokens en una lista de índices se denomina comúnmente *numeralización*.

El lote de entrada luego se pasa a través de la capa de embedding lo que nos da una representación vectorial densa de nuestras oraciones. `embedded` es un tensor de tamaño **[longitud de la oración, tamaño del lote, embedding_dim]**.

`embedded` se introduce luego en el RNN. En algunos frameworks, debemos alimentar el estado oculto inicial, $ h_0 $, en el RNN. Sin embargo, en PyTorch, si no se pasa un estado oculto inicial como argumento, se establece de forma predeterminada en un tensor de todos ceros.

El RNN devuelve 2 tensores, `output` de tamaño **[longitud de la oración, tamaño de lote, dimensión de la variable oculta]** y` hidden` de tamaño **[1, tamaño de lote, dimensión de la variable oculta]**. `output` es la concatenación del estado oculto de cada paso de tiempo, mientras que ` hidden` es simplemente el estado oculto final. Verificamos esto usando la declaración `assert`. Tenga en cuenta el método `squeeze`, que se utiliza para eliminar una dimensión de tamaño 1.

Finalmente, alimentamos el último estado oculto, "hidden", a través de la capa lineal, "fc", para producir una predicción.

In [ ]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        #########################################################################
        #TO_DO: Configure las capas que necesita para la RNN con la arquitectura#
        #       definida anteriormente.                                         #
        #########################################################################
        # ***** INICIO DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) ***** 
        super().__init__()

        self.embed = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.linear = nn.Linear(hidden_dim, output_dim)


        # ***** FINAL DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
        ########################################################################
        #                          FINAL DE TU CÓDIGO                          #       
        ########################################################################
        
    def forward(self, text):
        ########################################################################
        # TODO: Implementá la función forward para la RNN. Deberías            #
        # usar las capas que definiste en __init__ y especificar la            #
        # conectividad de dichas capas                                         #
        ########################################################################
        # ***** INICIO DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****

        #text = [sent len, batch size]
        #embedded = [sent len, batch size, emb dim]
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        x = self.embed(text)
        output , hidden = self.rnn(x)
        hidden = torch.squeeze(hidden)
        x = self.linear(hidden)
        return x 

        # ***** FINAL DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
        ########################################################################
        #                          FINAL DE TU CÓDIGO                          #       
        ########################################################################


Ahora creamos una instancia de nuestra clase RNN.

La dimensión de entrada es la dimensión de los vectores one-hot, que es igual al tamaño del vocabulario.

La dimensión de embedding es el tamaño de los vectores de palabras densas. Suele tener entre 50 y 250 dimensiones, pero depende del tamaño del vocabulario.

La dimensión oculta es el tamaño de las variables ocultas. Suele rondar entre 100 y 500 dimensiones, pero también depende de factores como el tamaño del vocabulario, el tamaño de los vectores densos y la complejidad de la tarea.

La dimensión de salida suele ser el número de clases, sin embargo, en el caso de solo 2 clases, el valor de salida está entre 0 y 1 y, por lo tanto, puede ser unidimensional, es decir, un solo número real escalar.

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

También creemos una función que nos diga cuántos parámetros entrenables tiene nuestro modelo para que podamos comparar el número de parámetros en diferentes modelos.

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'El modelo tiene {count_parameters(model):,} parámetros entrenables')

El modelo tiene 2,592,105 parámetros entrenables


## Entrenar el modelo

Ahora vamos a configurar el entrenamiento y a entrenar el modelo.

Primero, crearemos un optimizador. Este es el algoritmo que usamos para actualizar los parámetros del módulo. Aquí, usaremos _descenso de gradiente estocástico_ (SGD). El primer argumento son los parámetros que serán actualizados por el optimizador y el segundo es la tasa de aprendizaje, es decir, cuánto cambiaremos los parámetros cuando hagamos una actualización de parámetros.

In [ ]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

A continuación, definiremos nuestra función de pérdida. 

La función de pérdida aquí es _entropía cruzada binaria con logits_.

Nuestro modelo genera actualmente un número real sin consolidar. Como nuestras etiquetas son 0 o 1, queremos restringir las predicciones a un número entre 0 y 1. Hacemos esto usando las funciones _sigmoidea_.

Luego usamos este escalar para calcular la pérdida usando entropía cruzada binaria.

La clase `BCEWithLogitsLoss` lleva a cabo los pasos de entropía cruzada sigmoidea y binaria.

In [ ]:
loss = nn.BCEWithLogitsLoss()

Usando `.to`, podemos ubicar al modelo y la función de pérdida en el GPU (si tenemos alguno). 

In [ ]:
model = model.to(device)
loss = loss.to(device)

Ya establecimos cómo la pérdida, sin embargo, tenemos que escribir nuestra función para calcular el accuracy.

Esta función primero alimenta las predicciones a través de una capa sigmoidea, reescalando los valores entre 0 y 1, luego los redondeamos al número entero más cercano. Esto redondea cualquier valor superior a 0,5 a 1 (un sentimiento positivo) y el resto a 0 (un sentimiento negativo).

Luego calculamos cuántas predicciones redondeadas son iguales a las etiquetas reales y la promediamos en todo el lote.

In [ ]:
a = torch.ones(4) - 0.49
a = torch.sigmoid(a)
a

tensor([0.6248, 0.6248, 0.6248, 0.6248])

In [22]:
def binary_accuracy(preds, y):
    """
    Devuelve el accuracy por lote, es decir, si obtiene 8/10 correctamente,
    esto devuelve 0.8, NO 8
    """
    ########################################################################
    # TODO: Implementá la función binary_accuracy()                        #
    ########################################################################
    # ***** INICIO DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****

    sigmod = torch.sigmoid(preds)
    res = torch.round(sigmod)
    return  torch.sum(res == y) / len(preds)
      
    #suma=0
    # for x in range(0,len(res)-1):
    #   if res[x] == y[x]:
    #     suma = suma + 1
    #   else:
    #     suma = suma
    
    # ***** FINAL DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
    ########################################################################
    #                          FINAL DE TU CÓDIGO                          #       
    ########################################################################

La función `train_epoch` itera sobre todos los ejemplos, un lote a la vez.

`model.train()` se usa para poner el modelo en "modo de entrenamiento", lo que activa _ dropout_ y _batch normalization_. Aunque no los estamos usando en este modelo, es una buena práctica incluirlo.

Para cada lote, primero ponemos a cero los gradientes. Cada parámetro en un modelo tiene un atributo "grad" que almacena el gradiente calculado. PyTorch no elimina automáticamente los gradientes calculados a partir del último cálculo de gradiente, por lo que deben ponerse a cero manualmente.

Luego alimentamos el lote de oraciones, `batch.text`, en el modelo. Es necesario usar la función `squeeze` ya que las predicciones tienen inicialmente la forma **[tamaño de lote, 1]**, y necesitamos eliminar la dimensión de tamaño 1 ya que PyTorch espera que la entrada de predicciones a nuestra función de pérdida tenga forma **[tamaño del lote]**.

Luego, la pérdida y el accuracy se calculan utilizando nuestras predicciones y las etiquetas.

Calculamos el gradiente de cada parámetro con `loss.backward()`, y luego actualizamos los parámetros usando los gradientes y el algoritmo de optimización con `optimizer.step()`.

La pérdida y el accuracy se acumulan a lo largo de la época. Finalmente, devolvemos la pérdida y el accuracy, promediadas a lo largo de la época.

Recordemos que al inicializar el Field `LABEL`, configuramos` dtype = torch.float`. Esto se debe a que TorchText establece que los tensores sean `LongTensor`s por defecto, sin embargo, nuestra función de pérdida espera que ambas entradas sean` FloatTensor`s. Establecer el `dtype` en` torch.float`, hizo esto por nosotros. El método alternativo para hacer esto sería realizar la conversión dentro de la función `train` pasando` batch.label.float() `en lugar de` batch.label` a la función de pérdida.

In [ ]:
def train_epoch(model, iterator, optimizer, loss):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()

    ########################################################################
    # TODO: Implementá la función train_epoch() siguiendo las              #
    #       instrucciones de arriba.                                       #
    ########################################################################
    # ***** INICIO DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****

    for x,y in iterator:
      optimizer.zero_grad()
      salida_modelo = model(x)
      perdida = loss(torch.squeeze(salida_modelo), y)
      accuracy = binary_accuracy(torch.squeeze(salida_modelo), y)
      perdida.backward()
      optimizer.step()
    
      epoch_acc = epoch_acc + accuracy
      epoch_loss = epoch_loss + perdida 


    # ***** FINAL DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
    ########################################################################
    #                          FINAL DE TU CÓDIGO                          #       
    ########################################################################
    
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

`evaluate_epoch` es parecida a `train_epoch`, con algunas modificaciones ya que no desea actualizar los parámetros al evaluar.

`model.eval()` pone el modelo en "modo de evaluación", esto apaga _ dropout_ y _batch normalization_. Nuevamente, no los usamos en este modelo, pero es una buena práctica incluirlos.

No se calculan gradientes en las operaciones de PyTorch dentro del bloque `with no_grad()`. Esto hace que se utilice menos memoria y acelera el cálculo.

El resto de la función es igual que `train`, con la eliminación de` optimizer.zero_grad() `,` loss.backward () `y` optimizer.step () `, ya que no actualizamos los parámetros del modelo cuando evaluamos. 

In [ ]:
def evaluate_epoch(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    ########################################################################
    # TODO: Implementá la función evaluate_epoch() siguiendo las           #
    #       instrucciones de arriba.                                       #
    ########################################################################
    # ***** INICIO DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****

    with torch.no_grad():
      for x, y in iterator:
        salida_modelo = model(x)
        perdida = loss(torch.squeeze(salida_modelo), y)
        accuracy = binary_accuracy(torch.squeeze(salida_modelo), y)
        epoch_acc = epoch_acc + accuracy
        epoch_loss = epoch_loss + perdida


    # ***** FINAL DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
    ########################################################################
    #                          FINAL DE TU CÓDIGO                          #       
    ########################################################################
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

También vamos a crear una función para decirnos cuánto tarda una época en entrenarse para comparar los tiempos de entrenamiento entre modelos.

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Luego entrenamos el modelo a través de múltiples épocas, una época es un pase completo a través de todos los ejemplos en los conjuntos de entrenamiento y validación.

En cada época, si la pérdida de validación es la mejor que hemos visto hasta ahora, guardaremos los parámetros del modelo y luego, una vez finalizado el entrenamiento, usaremos ese modelo en el conjunto de prueba.

In [ ]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train_epoch(model, train_iterator, optimizer, loss)
    valid_loss, valid_acc = evaluate_epoch(model, valid_iterator, loss)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 14s
	Train Loss: 0.693 | Train Acc: 49.18%
	 Val. Loss: 0.693 |  Val. Acc: 49.70%
Epoch: 02 | Epoch Time: 0m 14s
	Train Loss: 0.693 | Train Acc: 49.08%
	 Val. Loss: 0.694 |  Val. Acc: 48.31%
Epoch: 03 | Epoch Time: 0m 14s
	Train Loss: 0.693 | Train Acc: 49.50%
	 Val. Loss: 0.693 |  Val. Acc: 49.71%
Epoch: 04 | Epoch Time: 0m 14s
	Train Loss: 0.693 | Train Acc: 49.41%
	 Val. Loss: 0.694 |  Val. Acc: 48.31%
Epoch: 05 | Epoch Time: 0m 14s
	Train Loss: 0.693 | Train Acc: 48.96%
	 Val. Loss: 0.694 |  Val. Acc: 48.32%


Es posible que haya notado que la pérdida no está disminuyendo realmente y la precisión es deficiente. Esto se debe a varios problemas con el modelo que mejoraremos en la siguiente sección.

Finalmente, la métrica que realmente nos importa, la pérdida de prueba y el accuracy, que obtenemos de nuestros parámetros que nos dieron la mejor pérdida de validación.

In [ ]:
model.load_state_dict(torch.load('tut1-model.pt'))


test_loss, test_acc = evaluate_epoch(model, test_iterator, loss)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.693 | Test Acc: 49.14%


# Sección 2 - Análisis de Sentimiento Potenciado


## Preparando los datos

Como antes, estableceremos la semilla, definiremos los `Fields` y obtendremos las divisiones entrenamiento/validación/prueba.

Usaremos **secuencias empaquetadas con padding**, lo que hará que nuestro RNN solo procese los tokens distintos a `<pad>` de nuestra secuencia, y para cualquier elemento `<pad>`, la "salida" será un tensor cero. Para usar secuencias empaquetadas con padding, tenemos que decirle al RNN la longitud de las secuencias reales. Hacemos esto configurando `include_lengths = True` para nuestro Field ` TEXT`. Esto hará que `batch.text` ahora sea una tupla, siendo el primer elemento nuestra oración (un tensor numérico que se ha rellenado) y el segundo elemento las longitudes reales de nuestras oraciones.

In [2]:
import torch
from torchtext.legacy import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm',
                  include_lengths = True)

LABEL = data.LabelField(dtype = torch.float)

Luego volvemos a cargar el dataset de IMDb.

In [3]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)


aclImdb_v1.tar.gz:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:01<00:00, 78.7MB/s]


Y volvemos a crear nuestro conjunto de validación a partir del de entrenamiento.



In [4]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

Lo siguiente es el uso de embeddings de palabras previamente entrenados. Ahora, en lugar de tener nuestras embeddings de palabras inicializados al azar, se inicializan con estos vectores pre-entrenados.
Obtenemos estos vectores simplemente especificando qué vectores queremos y pasándolos como argumento a `build_vocab`. `TorchText` se encarga de descargar los vectores y asociarlos con las palabras correctas en nuestro vocabulario.

Aquí, usaremos los vectores "glove.6B.100d". "Glove" es el algoritmo usado para calcular los vectores, ve [aquí](https://nlp.stanford.edu/projects/glove /) para más. "6B" indica que estos vectores se entrenaron en 6 mil millones de tokens y "100d" indica que estos vectores son 100-dimensionales.

Puede ver los otros vectores disponibles [aquí](https://github.com/pytorch/text/blob/master/torchtext/vocab.py#L113).

La teoría es que estos vectores pre-entrenados ya tienen palabras con un significado semántico similar juntas en el espacio vectorial, p. Ej. "terrible", "horrible", "espantoso" están cerca. Esto le da a nuestra capa de embedding una buena inicialización, ya que no tiene que aprender estas relaciones desde cero.

**Nota**: estos vectores pesan aproximadamente 862 MB, así que tenga cuidado si tiene una conexión a Internet limitada.

Por defecto, TorchText inicializará con cero aquellas palabras que estén en su vocabulario, pero no en los embeddings pre-entrenados. Esto no es deseable, por lo que en su lugar los inicializamos aleatoriamente configurando `unk_init` en` torch.Tensor.normal_`. Esto ahora inicializará esas palabras a través de una distribución gaussiana.

In [5]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [02:40, 5.39MB/s]                          
100%|█████████▉| 398178/400000 [00:14<00:00, 26812.79it/s]

Como antes, creamos los iteradores, colocando los tensores en la GPU si hay uno disponible.

Otra cosa para las secuencias empaquetadas con padding, todos los tensores dentro de un lote deben ordenarse según su longitud. Esto se maneja en el iterador configurando `sort_within_batch = True`.

In [6]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

## Construcción del Modelo
El modelo presenta los cambios más drásticos.

###  Una arquitectura RNN diferente

Usaremos una arquitectura RNN diferente llamada Long Short-Term Memory (LSTM). ¿Por qué una LSTM es mejor que una RNN estándar? Los RNN estándar sufren el [problema del gradiente que se desvanece](https://en.wikipedia.org/wiki/Vanishing_gradient_problem). Los LSTM superan esto al tener un estado recurrente adicional llamado **_célda_**, $c$, que puede considerarse como la "memoria" del LSTM, y el uso de múltiples **_compuertas_** que controlan el flujo de información dentro y fuera de la memoria. Para obtener más información, vaya [aquí](https://colah.github.io/posts/2015-08-Understanding-LSTMs/). 

Simplemente podemos pensar en el LSTM como una función de $ x_t $, $ h_t $ y $ c_t $, en lugar de solo $ x_t $ y $ h_t $.

$$ (h_t, c_t) = \text {LSTM} (x_t, h_t, c_t) $$

Por lo tanto, el modelo que usa un LSTM se parece a (con las capas de incrustación omitidas):

![](https://i.imgur.com/a8Qaw07.png)

El estado inicial de la celda, $ c_0 $, al igual que la variable oculta inicial, se inicializa a un tensor de todos ceros. Sin embargo, la predicción del sentimiento todavía se realiza utilizando la variable oculta final, no el estado final de la celda, es decir, $ \hat {y} = f (h_T) $.

###  RNN Bidireccional

El concepto detrás de una RNN bidireccional es simple. Además de tener una RNN procesando las palabras en la oración desde la primera hasta la última (una RNN hacia adelante), tenemos una segunda RNN procesando las palabras en la oración desde **la última a la primera** (una RNN hacia atrás) . En el tiempo $ t $, la RNN hacia adelante está procesando la palabra $ x_t $, y la RNN hacia atrás está procesando la palabra $ x_{T-t + 1} $.

En PyTorch, los tensores de las variables ocultas (y los estados de celda) devueltos por las RNN hacia adelante y hacia atrás se apilan uno encima del otro en un solo tensor.

Hacemos nuestra predicción de sentimiento usando una concatenación del último estado oculto de la RNN hacia adelante (obtenido de la palabra final de la oración), $ h_T ^ \rightarrow $, y el último estado oculto de la RNN hacia atrás (obtenido a partir de la primera palabra de la oración), $ h_T ^ \leftarrow $, es decir, $ \hat {y} = f (h_T ^ \rightarrow, h_T ^ \leftarrow) $

La siguiente imagen muestra una RNN bidireccional, con la RNN hacia adelante en naranja, la RNN hacia atrás en verde y la capa lineal en plateado.

![](https://i.imgur.com/g1mAJXt.png)

### RNN Multicapa

Las RNN multicapa (también llamados *RNN profundas*) son otro concepto simple. La idea es que agreguemos RNN adicionales sobre la RNN estándar inicial, donde cada RNN agregado es otra *capa*. La salida de la variable oculta de la primera RNN (inferior) en el tiempo $ t $ será la entrada de la RNN por encima de ella en el paso de tiempo $ t $. Luego, la predicción se realiza a partir de la variable oculta final de la capa final (más alta).

La siguiente imagen muestra una RNN unidireccional multicapa, donde el número de capa se da como superíndice. También tenga en cuenta que cada capa necesita su propia variable oculta inicial, $ h_0 ^ L $.

![](https://i.imgur.com/wBKSBhx.png)

### Regularización

Aunque hemos agregado mejoras a nuestro modelo, cada una de ellas agrega parámetros adicionales. Cuantos más parámetros tenga en su modelo, mayor será la probabilidad de que su modelo se sobreajuste (memorice los datos de entrenamiento, lo que provoca un error de entrenamiento bajo pero un error de validación/prueba alto, es decir, una generalización deficiente para ejemplos nuevos e inéditos). 

Para combatir esto, usamos la regularización. Más específicamente, usamos un método de regularización llamado **dropout**. El dropout funciona al *eliminar* (configurando en 0) neuronas en una capa al azar durante un pase hacia adelante. La probabilidad de que se descarte cada neurona se establece mediante un hiperparámetro y cada neurona con dropout aplicado se considera de forma independiente. Una teoría sobre por qué funciona el dropout es que un modelo con parámetros abandonados puede verse como un modelo "más simple" (menos parámetros). Las predicciones de todos estos modelos "más simples" (una para cada pase hacia adelante) se promedian juntas dentro de los parámetros del modelo. Por lo tanto, su único modelo puede considerarse como un conjunto de modelos más simples, ninguno de los cuales está sobre parametrizado y, por lo tanto, no debe sobreajustarse.

### Detalles de Implementación

Otra adición a este modelo es que no vamos a aprender el embedding del token `<pad>`. Esto se debe a que queremos decirle explícitamente a nuestro modelo que los tokens de relleno son irrelevantes para determinar el sentimiento de una oración. Esto significa que el embedding del token de padding permanecerá como se inicializó (todos ceros). Hacemos esto pasando el índice de nuestro token `<pad>` como el argumento `padding_idx` a la capa` nn.Embedding`.

Para usar un LSTM en lugar del RNN estándar, usamos `nn.LSTM` en lugar de` nn.RNN`. Además, tenga en cuenta que el LSTM devuelve la "salida" y una tupla con el estado final de la variable oculta y de la "celda", mientras que el RNN estándar solo devuelve la "salida" y el estado final de la variable oculta.

Como el estado oculto final de nuestro LSTM tiene un componente hacia adelante y hacia atrás, que se concatenarán juntos, el tamaño de la entrada a la capa `nn.Linear` es el doble que el tamaño de la dimensión oculta.

La implementación de la bidireccionalidad y la adición de capas adicionales se realizan pasando valores para los argumentos `num_layers` y` bidirectional` para el LSTM.

El dropout se implementa inicializando una capa `nn.Dropout` (el argumento es la probabilidad de descartar cada neurona) y usándola dentro del método` forward` después de cada capa a la que queremos aplicar el dropout. **Nota**: nunca use dropout en las capas de entrada o salida (`text` o` fc` en este caso), solo desea usarlo en las capas intermedias. Las LSTM tienes un argumento de "dropout" que añade dropout en las conexiones entre las variables ocultas en una capa y las variables ocultas en la siguiente capa.

Como estamos pasando las longitudes de nuestras oraciones para poder usar secuencias empaquetadas con padding, tenemos que agregar un segundo argumento, `text_lengths`, a` forward`.

Antes de pasar nuestras embeddings al RNN, necesitamos empaquetarlas, lo que hacemos con `nn.utils.rnn.packed_padded_sequence`. Esto hará que nuestro RNN solo procese los elementos no rellenados de nuestra secuencia. Entonces, el RNN devolverá `package_output` (una secuencia empaquetada) así como los estados` hidden` y `cell` (ambos son tensores). Sin secuencias empaquetadas con padding, ` hidden` y `cell` son tensores del último elemento de la secuencia, que probablemente será un token de relleno; sin embargo, cuando se utilizan secuencias empaquetadas con padding, ambos son del último elemento no rellenado de la secuencia. Tenga en cuenta que el argumento `lengths` de` package_padded_sequence` debe ser un tensor de CPU, por lo que lo convertimos explícitamente en uno usando `.to('cpu')`.

Luego descomprimimos la secuencia de salida, con `nn.utils.rnn.pad_packed_sequence`, para transformarla de una secuencia empaquetada a un tensor. Los elementos de "salida" de los tokens de relleno serán tensores cero (tensores donde cada elemento es cero). 

El estado final de la variable oculta, `hidden`, tiene una forma de **[núm capas * núm direcciones, tamaño de lote, dimensión de la variable oculta]**. Estos están ordenados: **[forward_layer_0, backward_layer_0, forward_layer_1, backward_layer 1, ..., forward_layer_n, backward_layer n]**. Como queremos que la capa final (superior) avance y retroceda las variables ocultas, obtenemos las dos capas superiores ocultas de la primera dimensión, `hidden [-2,:,:]` y `hidden [-1,:,:]` y las concatenamos antes de pasarlas a la capa lineal (después de aplicar dropout).

In [7]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 dropout, pad_idx, bidirectional, bias, batch_first ):
        #########################################################################
        #TO_DO: Configure las capas que necesita para la LSTM bidireccional     #
        #       multicapa con regularización definida anteriormente.            #
        #########################################################################
        # ***** INICIO DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) ***** 
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, pad_idx)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, bias, batch_first, dropout, bidirectional )
        self.linear = nn.Linear(hidden_dim*2, output_dim)
        self.dropout_ = nn.Dropout(dropout)
        

        # ***** FINAL DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
        ########################################################################
        #                          FINAL DE TU CÓDIGO                          #       
        ########################################################################
        
    def forward(self, text, text_lengths):
        ########################################################################
        # TODO: Implementá la función forward para la LSTM. Deberías           #
        # usar las capas que definiste en __init__ y especificar la            #
        # conectividad de dichas capas siguiendo los detalles de implementación#
        # definidos arriba                                                     #
        ########################################################################
        # ***** INICIO DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****

        #text = [sent len, batch size]
        
        #embedded = [sent len, batch size, emb dim]

        #empaquetamiento de la secuencia

        #output = [sent len, batch size, hid dim * num directions]
        #la salida de los tokens de padding son tensores con ceros
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concatenación de las variables ocultas hacia adelante y hacia atrás
        #y aplicar dropout

        #hidden = [batch size, hid dim * num directions]
        
        
        x = self.embedding(text)
        x = torch.nn.utils.rnn.pack_padded_sequence(x, text_lengths.to('cpu') )
        package_output, tupla = self.lstm(x)
        x , x_len =torch.nn.utils.rnn.pad_packed_sequence(package_output)
        hidden = tupla[0]
        hidden_final = torch.cat((hidden [-2,:,:], hidden [-1,:,:]), 1)
        hidden_final = self.dropout_(hidden_final)
        #hidden_final = hidden_final.reshape(-1, 64*256*2)
        x_final = self.linear(hidden_final)
        x_final = x_final.squeeze(1)
        
        return x_final
        

        # ***** FINAL DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
        ########################################################################
        #                          FINAL DE TU CÓDIGO                          #       
        ########################################################################
        


Como antes, crearemos una instancia de nuestra clase RNN, con los nuevos parámetros y argumentos para el número de capas, bidireccionalidad y probabilidad de dropout.

Para garantizar que los vectores entrenados previamente se puedan cargar en el modelo, el `EMBEDDING_DIM` debe ser igual al de los vectores GloVe previamente entrenados cargados anteriormente.

Obtenemos nuestro índice de token de pad del vocabulario, obteniendo la cadena real que representa el token de pad del atributo `pad_token` del campo, que es` <pad> `por defecto.

In [8]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
BIAS = True
BATCH_SIZE= False
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS,  
            DROPOUT, 
            PAD_IDX, 
            BIDIRECTIONAL,
            BIAS, 
            BATCH_SIZE)

Vamos a imprimir el número de parámetros en nuestro modelo.

¡Fijate cómo tenemos casi el doble de parámetros que antes!

In [9]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'El modelo tiene {count_parameters(model):,} parámetros entrenables')

El modelo tiene 4,810,857 parámetros entrenables


La última adición es copiar los embeddinga de palabras previamente entrenados en la capa "embeddings" de nuestro modelo.

Recuperamos los embeddings del vocabulario del Field y verificamos que sean del tamaño correcto,  **_[tamaño de vocabulario, dimensión de los embeddings]_**.

In [10]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


Luego, reemplazamos los pesos iniciales de la capa de "embedding" con los embeddings pre-entrenados.

**Nota**: ¡esto siempre debe hacerse en el `weight.data` y no en el` weight`!

In [11]:
#inserte el código aquí para inicializar con los embeddings preentrenados
model.embedding.weight.data.copy_(pretrained_embeddings)
model.embedding.weight.requires_grad = False

Como nuestros tokens `<unk>` y `<pad>` no están en el vocabulario pre-entrenado, han sido inicializados usando al construir nuestro vocabulario `unk_init` (una distribución $ \mathcal {N} (0,1) $). Es preferible inicializar ambos a todos ceros para decirle explícitamente a nuestro modelo que, inicialmente, son irrelevantes para determinar el sentimiento.

Hacemos esto estableciendo manualmente su fila en la matriz de pesos de incrustación en ceros. Obtenemos su fila encontrando el índice de los tokens, lo que ya hemos hecho para el índice de relleno.

**Nota**: al igual que al inicializar los embeddings, ¡esto debe hacerse en el `weight.data` y no en el` weight`!

In [12]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

#inserte su código aquí para inicializar con ceros los tokens <unk> y <pad> 
model.embedding.weight.data[UNK_IDX, :] = 0
model.embedding.weight.data[PAD_IDX, :] = 0


print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.3335,  0.0274,  0.1259,  ..., -0.0510,  0.3407,  0.6567],
        [-0.0734,  0.0609, -0.5752,  ...,  0.3456,  0.3631, -0.2070],
        [-0.1278,  0.0041,  1.0966,  ...,  0.1481,  0.7298, -0.3969]])


Ahora podemos ver que las dos primeras filas de la matriz de pesos de incrustación se han establecido en ceros. A medida que pasamos el índice del token de pad al `padding_idx` de la capa de embedding, este permanecerá en ceros durante el entrenamiento, sin embargo, sí se aprenderá el embedding del token `<unk>`.

## Entrenamiento del Modelo

Ahora a entrenar el modelo.

El único cambio que vamos a hacer acá es cambiar el optimizador de "SGD" a "Adam". SGD actualiza todos los parámetros con la misma tasa de aprendizaje y elegir esta tasa de aprendizaje puede ser complicado. `Adam` adapta la tasa de aprendizaje para cada parámetro, dandole a los parámetros que se actualizan con mayor frecuencia tasas de aprendizaje más bajas y a los parámetros que se actualizan con poca frecuencia, tasas de aprendizaje más altas. Puede encontrar más información sobre `Adam` (y otros optimizadores) [aquí](http://ruder.io/optimizing-gradient-descent/index.html).

Para cambiar `SGD` a` Adam`, simplemente cambiamos `optim.SGD` a` optim.Adam`, también tenga en cuenta que no tenemos que proporcionar una tasa de aprendizaje inicial para Adam, ya que PyTorch especifica una tasa de aprendizaje inicial predeterminada sensible.

In [13]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

El resto de los pasos para entrenar el modelo no se modifican.

Definimos la función de pérdida y la colocamos junto con el modelo en la GPU (si está disponible)...

In [14]:
loss = nn.BCEWithLogitsLoss()

model = model.to(device)
loss = loss.to(device)

Definimos una función para entrenar nuestro modelo.

Como hemos establecido `include_lengths = True`, nuestro ` batch.text` ahora es una tupla, siendo el primer elemento el tensor numérico y el segundo elemento las longitudes reales de cada secuencia. Los separamos en sus propias variables, `text` y` text_lengths`, antes de pasarlos al modelo.

**Nota**: como ahora estamos usando dropout, debemos recordar usar `model.train()` para asegurarnos de que el dropout esté "activado" durante el entrenamiento.

In [20]:
def train_epoch(model, iterator):
    
    epoch_loss = 0
    epoch_acc = 0

    model.train()
    ########################################################################
    # TODO: Implementá la función train_epoch() siguiendo las              #
    #       instrucciones de arriba teniendo en cuenta las secuencias      #
    #       empaquetadas con padding.                                      #
    ########################################################################
    # ***** INICIO DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****

    for x,y in iterator:
      optimizer.zero_grad()
      text, text_length = x
      salida_modelo = model(text, text_length)
      #print(salida_modelo)
      #print(y)
      perdida = loss(salida_modelo, y)
      accuracy = binary_accuracy(salida_modelo, y)
      perdida.backward()
      optimizer.step()
    
      epoch_acc = epoch_acc + accuracy 
      epoch_loss = epoch_loss + perdida 

    # ***** FINAL DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
    ########################################################################
    #                          FINAL DE TU CÓDIGO                          #       
    ########################################################################
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Luego definimos una función para probar nuestro modelo, recordando de nuevo separar `batch.text`.

**Nota**: como ahora estamos usando dropout, debemos recordar usar `model.eval()` para asegurarnos de que el dropout esté "desactivado" durante la evaluación.

In [16]:
def evaluate_epoch(model, iterator):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    ########################################################################
    # TODO: Implementá la función evaluate_epoch() siguiendo las           #
    #       instrucciones de arriba teniendo en cuenta las secuencias      #
    #       empaquetadas con padding.                                      #
    ########################################################################
    # ***** INICIO DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****

    with torch.no_grad():
      for x, y in iterator:
        text, text_length = x
        salida_modelo = model(text, text_length)
        perdida = loss(salida_modelo, y)
        accuracy = binary_accuracy(salida_modelo, y)
        epoch_acc = epoch_acc + accuracy
        epoch_loss = epoch_loss + perdida

    # ***** FINAL DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
    ########################################################################
    #                          FINAL DE TU CÓDIGO                          #       
    ########################################################################
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Y también crea una función bonita para decirnos cuánto tardan en ejecutarse nuestras épocas.

In [17]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Finalmente, entrenamos a nuestro modelo...

In [23]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train_epoch(model, train_iterator)
    valid_loss, valid_acc = evaluate_epoch(model, valid_iterator)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 36s
	Train Loss: 0.671 | Train Acc: 58.33%
	 Val. Loss: 0.644 |  Val. Acc: 63.07%
Epoch: 02 | Epoch Time: 0m 37s
	Train Loss: 0.651 | Train Acc: 61.29%
	 Val. Loss: 0.586 |  Val. Acc: 68.68%
Epoch: 03 | Epoch Time: 0m 38s
	Train Loss: 0.602 | Train Acc: 66.99%
	 Val. Loss: 0.579 |  Val. Acc: 68.82%
Epoch: 04 | Epoch Time: 0m 38s
	Train Loss: 0.581 | Train Acc: 69.44%
	 Val. Loss: 0.615 |  Val. Acc: 65.61%
Epoch: 05 | Epoch Time: 0m 39s
	Train Loss: 0.588 | Train Acc: 67.95%
	 Val. Loss: 0.548 |  Val. Acc: 72.36%


... ¡y obtenemos nuestra nueva y mejorada precisión de prueba! 

In [24]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate_epoch(model, test_iterator)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.547 | Test Acc: 72.49%


# Sección 3 - Predicciones con entrada del usuario

Ahora podemos usar nuestro modelo para predecir el sentimiento de cualquier oración que le demos. Como ha sido entrenado en reseñas de películas, las oraciones proporcionadas también deben ser reseñas de películas.

Cuando se usa un modelo para la predicción, siempre debe estar en modo de evaluación. Si se sigue este tutorial paso a paso, entonces ya debería estar en modo de evaluación (desde que ejecutamos `evaluate()` en el conjunto de prueba), sin embargo, lo configuramos explícitamente para evitar cualquier riesgo.

Nuestra función `predict_sentiment` debe hacer algunas cosas:
- configurar el modelo en modo de evaluación
- tokenizar la oración, es decir, pasar de una cadena sin procesar a una lista de tokens
- indexar los tokens convirtiéndolos en su representación entera de nuestro vocabulario
- obtener la longitud de nuestra secuencia
- convertir los índices, que son una lista de Python en un tensor de PyTorch
- agregar una dimensión de lote al hacer `unsqueeze`
- convertir la longitud de la secuencia en un tensor
- rescalar la predicción de salida a un rango entre 0 y 1 con la función "sigmoide"
- convertir el tensor que contiene un valor único en un número entero con el método `item()`

Esperamos que las reseñas con un sentimiento negativo devuelvan un valor cercano a 0 y que las reseñas positivas devuelvan un valor cercano a 1.

In [55]:
a=4
torch.Tensor([a])

tensor([4.])

In [89]:
import spacy
nlp = spacy.load('en_core_web_sm')

def predict_sentiment(model, sentence):
    ########################################################################
    # TODO: Implementá la función predict_sentiment() siguiendo las        #
    #       instrucciones de arriba.                                       #
    ########################################################################
    # ***** INICIO DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****

    model.eval()
    oracion_tokens = nlp(sentence)
    oracion_tokens = [oracion.text for oracion in oracion_tokens]
    
    oracion_indices = [TEXT.vocab.stoi[i] for i in oracion_tokens]
    
    longitud_secuencia = len(oracion_indices)
    
    tensor = torch.IntTensor(oracion_indices)
    
    tensor = tensor.unsqueeze(1)
    
    longitud_tensor = torch.IntTensor([longitud_secuencia])
    
    salida = model(tensor.to(device), longitud_tensor.to(device))
    
    salida = torch.sigmoid(salida)
    
    return salida.item()


  
    # ***** FINAL DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
    ########################################################################
    #                          FINAL DE TU CÓDIGO                          #       
    ########################################################################

Un ejemplo de reseña negativa...

In [90]:
predict_sentiment(model, "This film is terrible")

0.08206340670585632

Un ejemplo de reseña positiva...

In [91]:
predict_sentiment(model, "This film is great")

0.879744291305542